# 根据要求导出`WRF-CMAQ`模拟数据
两大剖面的数据。用matlab存成三维数组，三个维度分别是水平位置、垂直高度和小时时刻，用这样的三维数组去装载各指标，包括**O3浓度、PM2.5浓度、NO2浓度、各过程贡献、各来源贡献**，例如O3浓度的三维数组是CO3（2点位,300米,4月2日13时）=75μg/m3。其中水平位置可以用1~n代表n个经纬度组合，再单独给一个对照表即可，例如2点位=（E 113.6°、N 29.1°）。

---
*@author: Evan*\
*@date: 2023-08-09*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.io import savemat

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../../src/')
from findpoint import findpoint
import ModelEvaLib as me
from namelist import *

In [2]:
lon1, lat1 = 116.6, 31.1
lon2, lat2 = 117.4, 30.1

lon3, lat3 = 116.1, 29.74
lon4, lat4 = 117.46, 30.85

lon5, lat5 = 116.8, 30.54
lon6, lat6 = 117.3, 30.5

In [3]:
chem = xr.open_dataset(cmaqfile)
met = xr.open_dataset(mcipfile)
pa = xr.open_dataset(pafile)
isam1 = xr.open_dataset(isamfile1)
isam2 = xr.open_dataset(isamfile2)

In [4]:
nlevel = 21
number = 50

lat = chem.latitude
lon = chem.longitude

O3   = chem.O3[:,:nlevel,:,:]
NO2  = chem.NO2[:,:nlevel,:,:]
PM25 = chem.PM25[:,:nlevel,:,:]
ht   = chem.HT[:,:nlevel,:,:]

# mda8   = O3.rolling(time=8).mean().resample({'time':'D'}).max(dim='time')
# NO2avg = NO2.resample({'time':'D'}).mean(dim='time')
# PMavg  = PM25.resample({'time':'D'}).mean(dim='time')
# htavg  = ht.resample({'time':'D'}).mean(dim='time')
# print(mda8.shape)

HADV = pa.HADV_O3[:,:nlevel,:,:]
ZADV = pa.ZADV_O3[:,:nlevel,:,:]
HDIF = pa.HDIF_O3[:,:nlevel,:,:]
VDIF = pa.VDIF_O3[:,:nlevel,:,:]
DDEP = pa.DDEP_O3[:,:nlevel,:,:]
CHEM = pa.CHEM_O3[:,:nlevel,:,:]

O3_AnQ = isam1.O3_AnQ[:,:nlevel,:,:]
O3_AQI = isam1.O3_AQI[:,:nlevel,:,:]
O3_AQT = isam1.O3_AQT[:,:nlevel,:,:]
O3_AQA = isam1.O3_AQA[:,:nlevel,:,:]
O3_AQP = isam1.O3_AQP[:,:nlevel,:,:]
O3_OTH = isam1.O3_OTH[:,:nlevel,:,:]
O3_BCO = isam1.O3_BCO[:,:nlevel,:,:]
O3_ICO = isam1.O3_ICO[:,:nlevel,:,:]

PM_AnQ = isam2.PM_AnQ[:,:nlevel,:,:]
PM_AQI = isam2.PM_AQI[:,:nlevel,:,:]
PM_AQT = isam2.PM_AQT[:,:nlevel,:,:]
PM_AQA = isam2.PM_AQA[:,:nlevel,:,:]
PM_AQP = isam2.PM_AQP[:,:nlevel,:,:]
PM_OTH = isam2.PM_OTH[:,:nlevel,:,:]
PM_BCO = isam2.PM_BCO[:,:nlevel,:,:]
PM_ICO = isam2.PM_ICO[:,:nlevel,:,:]

HNO3 = pa.HNO3prod[:,:nlevel,:,:]
H2O2 = pa.H2O2prod[:,:nlevel,:,:]

u = met.uwind[:,:nlevel,:,:]
v = met.vwind[:,:nlevel,:,:]
w = chem.wwind[:,:nlevel,:,:]

## Line2

In [5]:
# 生成沿两个点连线的坐标
lons = np.linspace(lon3,lon4,num=number)
lats = np.linspace(lat3,lat4,num=number)

# 选取这些点在数据集中的最接近的网格点的数据
x_index = []
y_index = []

for i in range(number): 
    out_x, out_y = findpoint(lons[i],lats[i],O3)
    x_index.append(out_x)
    y_index.append(out_y)
O3_sel   = O3.isel(x=x_index, y=y_index)
NO2_sel  = NO2.isel(x=x_index, y=y_index)
PM25_sel = PM25.isel(x=x_index, y=y_index)
htsel    = ht.isel(x=x_index, y=y_index)

HADV_sel  = HADV.isel(x=x_index, y=y_index)
ZADV_sel  = ZADV.isel(x=x_index, y=y_index)
HDIF_sel  = HDIF.isel(x=x_index, y=y_index)
VDIF_sel  = VDIF.isel(x=x_index, y=y_index)
DDEP_sel  = DDEP.isel(x=x_index, y=y_index)
CHEM_sel  = CHEM.isel(x=x_index, y=y_index)

O3AnQ_sel = O3_AnQ.isel(x=x_index, y=y_index)
O3AQI_sel = O3_AQI.isel(x=x_index, y=y_index)
O3AQT_sel = O3_AQT.isel(x=x_index, y=y_index)
O3AQA_sel = O3_AQA.isel(x=x_index, y=y_index)
O3AQP_sel = O3_AQP.isel(x=x_index, y=y_index)
O3OTH_sel = O3_OTH.isel(x=x_index, y=y_index)
O3ICO_sel = O3_ICO.isel(x=x_index, y=y_index)
O3BCO_sel = O3_BCO.isel(x=x_index, y=y_index)

PMAnQ_sel = PM_AnQ.isel(x=x_index, y=y_index)
PMAQI_sel = PM_AQI.isel(x=x_index, y=y_index)
PMAQT_sel = PM_AQT.isel(x=x_index, y=y_index)
PMAQA_sel = PM_AQA.isel(x=x_index, y=y_index)
PMAQP_sel = PM_AQP.isel(x=x_index, y=y_index)
PMOTH_sel = PM_OTH.isel(x=x_index, y=y_index)
PMICO_sel = PM_ICO.isel(x=x_index, y=y_index)
PMBCO_sel = PM_BCO.isel(x=x_index, y=y_index)

HNO3_sel = HNO3.isel(x=x_index, y=y_index)
H2O2_sel = H2O2.isel(x=x_index, y=y_index)

u_sel = u.isel(x=x_index, y=y_index)
v_sel = v.isel(x=x_index, y=y_index)
w_sel = w.isel(x=x_index, y=y_index)


In [6]:
timestep=744
diag_O3 = np.zeros([timestep,21,number])
diag_NO2 = np.zeros([timestep,21,number])
diag_PM25 = np.zeros([timestep,21,number])
diag_ht = np.zeros([timestep,21,number])

diag_HADV = np.zeros([timestep,21,number])
diag_ZADV = np.zeros([timestep,21,number])
diag_HDIF = np.zeros([timestep,21,number])
diag_VDIF = np.zeros([timestep,21,number])
diag_DDEP = np.zeros([timestep,21,number])
diag_CHEM = np.zeros([timestep,21,number])

diag_O3AnQ = np.zeros([timestep,21,number])
diag_O3AQI = np.zeros([timestep,21,number])
diag_O3AQT = np.zeros([timestep,21,number])
diag_O3AQA = np.zeros([timestep,21,number])
diag_O3AQP = np.zeros([timestep,21,number])
diag_O3OTH = np.zeros([timestep,21,number])
diag_O3ICO = np.zeros([timestep,21,number])
diag_O3BCO = np.zeros([timestep,21,number])

diag_PMAnQ = np.zeros([timestep,21,number])
diag_PMAQI = np.zeros([timestep,21,number])
diag_PMAQT = np.zeros([timestep,21,number])
diag_PMAQA = np.zeros([timestep,21,number])
diag_PMAQP = np.zeros([timestep,21,number])
diag_PMOTH = np.zeros([timestep,21,number])
diag_PMICO = np.zeros([timestep,21,number])
diag_PMBCO = np.zeros([timestep,21,number])

diag_HNO3 = np.zeros([timestep,21,number])
diag_H2O2 = np.zeros([timestep,21,number])
diag_u = np.zeros([timestep,21,number])
diag_v = np.zeros([timestep,21,number])
diag_w = np.zeros([timestep,21,number])

In [7]:
for i in range(number):
    diag_O3[:,:,i] = O3_sel[:,:,i,i]
    diag_NO2[:,:,i] = NO2_sel[:,:,i,i]
    diag_PM25[:,:,i] = PM25_sel[:,:,i,i]
    diag_ht[:,:,i] = htsel[:,:,i,i]
    
    diag_HADV[:,:,i] = HADV_sel[:,:,i,i]
    diag_ZADV[:,:,i] = ZADV_sel[:,:,i,i]
    diag_HDIF[:,:,i] = HDIF_sel[:,:,i,i]
    diag_VDIF[:,:,i] = VDIF_sel[:,:,i,i]
    diag_DDEP[:,:,i] = DDEP_sel[:,:,i,i]
    diag_CHEM[:,:,i] = CHEM_sel[:,:,i,i]
    
    diag_O3AnQ[:,:,i] = O3AnQ_sel[:,:,i,i]
    diag_O3AQI[:,:,i] = O3AQI_sel[:,:,i,i]
    diag_O3AQT[:,:,i] = O3AQT_sel[:,:,i,i]
    diag_O3AQA[:,:,i] = O3AQA_sel[:,:,i,i]
    diag_O3AQP[:,:,i] = O3AQP_sel[:,:,i,i]
    diag_O3OTH[:,:,i] = O3OTH_sel[:,:,i,i]
    diag_O3ICO[:,:,i] = O3ICO_sel[:,:,i,i]
    diag_O3BCO[:,:,i] = O3BCO_sel[:,:,i,i]
    
    diag_PMAnQ[:,:,i] = PMAnQ_sel[:,:,i,i]
    diag_PMAQI[:,:,i] = PMAQI_sel[:,:,i,i]
    diag_PMAQT[:,:,i] = PMAQT_sel[:,:,i,i]
    diag_PMAQA[:,:,i] = PMAQA_sel[:,:,i,i]
    diag_PMAQP[:,:,i] = PMAQP_sel[:,:,i,i]
    diag_PMOTH[:,:,i] = PMOTH_sel[:,:,i,i]
    diag_PMICO[:,:,i] = PMICO_sel[:,:,i,i]
    diag_PMBCO[:,:,i] = PMBCO_sel[:,:,i,i]
    
    diag_HNO3[:,:,i] = HNO3_sel[:,:,i,i]
    diag_H2O2[:,:,i] = H2O2_sel[:,:,i,i]
    
    diag_u[:,:,i] = u_sel[:,:,i,i]
    diag_v[:,:,i] = v_sel[:,:,i,i]
    diag_w[:,:,i] = w_sel[:,:,i,i]


In [8]:
Sens = diag_H2O2/diag_HNO3
Sens.shape

(744, 21, 50)

In [9]:
df_line1 = pd.DataFrame(
    index=np.arange(1,51),
    data={'经度':lons,'纬度':lats}
)
df_line1.to_excel('D:/Download/line2.xlsx')

In [10]:
savemat(
    'D:/Download/AQ_2307_line2.mat',
    {
        'height':diag_ht,
        'O3':diag_O3,
        'NO2':diag_NO2,
        'PM25':diag_PM25,
        'HADV':diag_HADV,
        'ZADV':diag_ZADV,
        'HDIF':diag_HDIF,
        'VDIF':diag_VDIF,
        'DDEP':diag_DDEP,
        'CHEM':diag_CHEM,
        'O3_AnQ':diag_O3AnQ,
        'O3_AQI':diag_O3AQI,
        'O3_AQT':diag_O3AQT,
        'O3_AQA':diag_O3AQA,
        'O3_AQP':diag_O3AQP,
        'O3_OTH':diag_O3OTH,
        'O3_BCO':diag_O3BCO,
        'O3_ICO':diag_O3ICO,
        'PM_AnQ':diag_PMAnQ,
        'PM_AQI':diag_PMAQI,
        'PM_AQT':diag_PMAQT,
        'PM_AQA':diag_PMAQA,
        'PM_AQP':diag_PMAQP,
        'PM_OTH':diag_PMOTH,
        'PM_BCO':diag_PMBCO,
        'PM_ICO':diag_PMICO,
        'Sensitivity':Sens,
        'u':diag_u,
        'v':diag_v,
        'w':diag_w,
    }
)